# Analýza sentimentu textu pomocí LSTM

Tento notebook implementuje model pro automatickou analýzu sentimentu textů pomocí hluboké neuronové sítě (LSTM).

**Cíl projektu:** Binární klasifikace sentimentu (pozitivní/negativní) na datasetu Sentiment140.

## Obsah
1. Nastavení prostředí a import knihoven
2. Načtení a explorace dat
3. Preprocessing textu
4. Tokenizace a příprava dat
5. Vytvoření LSTM modelu
6. Trénování modelu
7. Vyhodnocení a vizualizace
8. Inference pro nové texty

## 1. Nastavení prostředí

Nejprve ověříme připojení ke Google Colab a zkontrolujeme dostupný hardware (GPU).

In [ ]:
!nvidia-smi

In [1]:
# Ověření prostředí - zjistíme, jestli běžíme na Colab a jaký máme hardware
import sys

# Kontrola, zda běžíme v Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Běžíme v Google Colab")
except ImportError:
    IN_COLAB = False
    print("⚠️ Neběžíme v Colab - používáme lokální prostředí")

# Kontrola dostupného GPU
import tensorflow as tf
print(f"\n📦 TensorFlow verze: {tf.__version__}")
print(f"🐍 Python verze: {sys.version}")

# Kontrola GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"🚀 GPU dostupné: {len(gpus)}")
    for gpu in gpus:
        print(f"   - {gpu.name}")
else:
    print("💻 GPU není dostupné - použijeme CPU")

### Import knihoven

Nyní naimportujeme všechny potřebné knihovny:
- **TensorFlow/Keras** - framework pro deep learning
- **NumPy** - numerické výpočty
- **Pandas** - práce s daty
- **scikit-learn** - metriky a rozdělení dat
- **Matplotlib** - vizualizace

In [ ]:
# Import všech potřebných knihoven
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re  # pro čištění textu (regulární výrazy)

# TensorFlow a Keras komponenty
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Scikit-learn pro metriky a rozdělení dat
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

# Nastavení pro reprodukovatelnost výsledků
np.random.seed(42)
tf.random.set_seed(42)

print("✅ Všechny knihovny úspěšně naimportovány!")

## 2. Načtení dat

Dataset **Sentiment140** obsahuje 1.6 milionu tweetů označených jako pozitivní (4) nebo negativní (0).

Stáhneme ho z Kaggle pomocí knihovny `kagglehub`.

In [ ]:
# Stažení datasetu Sentiment140 z Kaggle
# Při prvním spuštění může být potřeba autentizace

import kagglehub

# Stažení datasetu - kagglehub ho uloží do cache
print("📥 Stahuji dataset Sentiment140...")
dataset_path = kagglehub.dataset_download("kazanova/sentiment140")
print(f"✅ Dataset stažen do: {dataset_path}")

In [ ]:
# Načtení datasetu do pandas DataFrame
import os

# Najdeme CSV soubor v datasetu
csv_file = os.path.join(dataset_path, "training.1600000.processed.noemoticon.csv")

# Dataset nemá hlavičky, musíme je definovat ručně
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

# Načtení CSV - encoding je latin-1 (ISO-8859-1)
print("📂 Načítám data do paměti...")
df = pd.read_csv(csv_file, encoding='latin-1', names=column_names)

print(f"✅ Načteno {len(df):,} záznamů")
print(f"📊 Sloupce: {list(df.columns)}")

### Explorace dat

Podíváme se, jak data vypadají — prvních pár řádků, rozložení sentimentu, příklady textů.

In [ ]:
# Zobrazení prvních 5 řádků
print("📋 Prvních 5 záznamů:")
df.head()



In [ ]:
# Rozložení sentimentu v datasetu
print("📊 Rozložení sentimentu:")
print(df['target'].value_counts())

# Vizualizace
fig, ax = plt.subplots(figsize=(6, 4))
df['target'].value_counts().plot(kind='bar', ax=ax, color=['#ff6b6b', '#51cf66'])
ax.set_xticklabels(['Negativní (0)', 'Pozitivní (4)'], rotation=0)
ax.set_ylabel('Počet')
ax.set_title('Rozložení sentimentu v datasetu')
plt.tight_layout()
plt.show()

In [ ]:
# Příklady negativních a pozitivních tweetů
print("😠 Příklady NEGATIVNÍCH tweetů:")
for text in df[df['target'] == 0]['text'].head(3):
    print(f"  • {text[:100]}...")

print("\n😊 Příklady POZITIVNÍCH tweetů:")
for text in df[df['target'] == 4]['text'].head(3):
    print(f"  • {text[:100]}...")

## 3. Preprocessing textu

Tweety obsahují mnoho "šumu", který model nepotřebuje:
- **@mentions** — uživatelská jména
- **URL odkazy** — http://...
- **Speciální znaky** — emotikony, HTML entity
- **Velká písmena** — převedeme na malá

Vytvoříme funkci pro čištění textu.

In [ ]:
def clean_text(text):
    """
    Vyčistí tweet od šumu.
    
    Args:
        text: Původní text tweetu
    Returns:
        Vyčištěný text
    """
    # Převod na malá písmena
    text = text.lower()
    
    # Odstranění @mentions (např. @user123)
    text = re.sub(r'@\w+', '', text)
    
    # Odstranění URL odkazů
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Odstranění HTML entit (např. &amp; &lt;)
    text = re.sub(r'&\w+;', '', text)
    
    # Odstranění speciálních znaků a číslic (ponecháme jen písmena a mezery)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Odstranění nadbytečných mezer
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Test funkce na příkladu
test_tweet = "@user123 I love this movie!!! 😊 Check it out: http://example.com #awesome"
print(f"Původní:   '{test_tweet}'")
print(f"Vyčištěný: '{clean_text(test_tweet)}'")

In [ ]:
# Aplikace čištění na celý dataset
# POZOR: Toto může trvat několik minut na 1.6M záznamech!

print("🧹 Čistím texty...")
df['clean_text'] = df['text'].apply(clean_text)
print("✅ Hotovo!")

# Ukázka před/po
print("\n📊 Ukázka vyčištěných textů:")
for i in range(3):
    print(f"\nPůvodní:   {df['text'].iloc[i][:80]}...")
    print(f"Vyčištěný: {df['clean_text'].iloc[i][:80]}...")

In [ ]:
# Převod sentimentu na binární hodnoty (0 = negativní, 1 = pozitivní)
# Původně: 0 = negativní, 4 = pozitivní
# Nově:    0 = negativní, 1 = pozitivní

df['sentiment'] = df['target'].apply(lambda x: 1 if x == 4 else 0)

print("📊 Nové rozložení sentimentu:")
print(df['sentiment'].value_counts())
print(f"\n0 = Negativní, 1 = Pozitivní")

## 4. Tokenizace a příprava dat

Neuronová síť nerozumí textu — potřebuje čísla. Proto musíme:

1. **Tokenizace** — každé slovo dostane unikátní číslo (index)
2. **Padding** — všechny sekvence musí mít stejnou délku

Příklad: `"i love this"` → `[1, 45, 23]` → `[1, 45, 23, 0, 0, 0, ...]`

In [ ]:
# Hyperparametry pro tokenizaci
VOCAB_SIZE = 20000      # Maximální počet slov ve slovníku
MAX_LENGTH = 100        # Maximální délka sekvence (tweetu)
EMBEDDING_DIM = 128     # Dimenze embedding vektoru

print(f"📊 Nastavení:")
print(f"   Velikost slovníku: {VOCAB_SIZE:,} slov")
print(f"   Max. délka tweetu: {MAX_LENGTH} tokenů")
print(f"   Embedding dimenze: {EMBEDDING_DIM}")

In [ ]:
# Vytvoření tokenizeru a trénování na našich datech
print("🔤 Vytvářím tokenizer a slovník...")

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
tokenizer.fit_on_texts(df['clean_text'])

# Informace o slovníku
word_index = tokenizer.word_index
print(f"✅ Slovník vytvořen!")
print(f"   Celkem unikátních slov: {len(word_index):,}")
print(f"   Použijeme top {VOCAB_SIZE:,} nejčastějších")

# Ukázka několika slov a jejich indexů
print("\n📖 Ukázka slovníku (slovo → index):")
for word, idx in list(word_index.items())[:10]:
    print(f"   '{word}' → {idx}")

In [ ]:
# Převod textů na sekvence čísel
print("🔢 Převádím texty na sekvence čísel...")

sequences = tokenizer.texts_to_sequences(df['clean_text'])

# Ukázka převodu
print("\n📝 Ukázka převodu text → sekvence:")
print(f"   Text: '{df['clean_text'].iloc[0][:50]}...'")
print(f"   Sekvence: {sequences[0][:10]}...")

In [ ]:
# Padding - doplnění/zkrácení na stejnou délku
print(f"📏 Aplikuji padding na délku {MAX_LENGTH}...")

X = pad_sequences(sequences, maxlen=MAX_LENGTH, padding='pre', truncating='post')
y = df['sentiment'].values

print(f"✅ Data připravena!")
print(f"   Tvar X (vstupy): {X.shape}")
print(f"   Tvar y (labely): {y.shape}")

# Ukázka paddované sekvence
print(f"\n📝 Ukázka paddované sekvence:")
print(f"   Délka: {len(X[0])}")
print(f"   Prvních 20 hodnot: {X[0][:20]}")

### Rozdělení dat

Rozdělíme data na 3 části:
- **Training (80%)** — pro trénování modelu
- **Validation (10%)** — pro sledování během trénování
- **Test (10%)** — pro finální vyhodnocení

In [ ]:
# Rozdělení na train a temp (test + validation)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, 
    test_size=0.2,      # 20% pro test+val
    random_state=42,    # Pro reprodukovatelnost
    stratify=y          # Zachová poměr tříd
)

# Rozdělení temp na validation a test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,      # 50% z 20% = 10% celkem
    random_state=42,
    stratify=y_temp
)

print("✅ Data rozdělena!")
print(f"   Train:      {X_train.shape[0]:,} vzorků ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"   Validation: {X_val.shape[0]:,} vzorků ({X_val.shape[0]/len(X)*100:.0f}%)")
print(f"   Test:       {X_test.shape[0]:,} vzorků ({X_test.shape[0]/len(X)*100:.0f}%)")

## 5. Vytvoření LSTM modelu

Architektura modelu:

```
Input (sekvence čísel) 
    ↓
Embedding (slova → vektory)
    ↓
LSTM (zachycení kontextu)
    ↓
Dropout (prevence přetrénování)
    ↓
Dense + Sigmoid (výstup 0-1)
```

In [ ]:
# Vytvoření LSTM modelu
model = Sequential([
    # Embedding vrstva - převede indexy slov na husté vektory
    # input_dim = velikost slovníku, output_dim = dimenze vektoru
    Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM),
    
    # LSTM vrstva - zachytí kontext a vztahy mezi slovy
    # 64 = počet LSTM jednotek (neuronů)
    LSTM(64, return_sequences=False),
    
    # Dropout - náhodně "vypne" 50% neuronů při trénování
    # Pomáhá proti overfittingu (přeučení)
    Dropout(0.5),
    
    # Výstupní vrstva - 1 neuron s sigmoid aktivací
    # Sigmoid vrací hodnotu 0-1 (pravděpodobnost pozitivního sentimentu)
    Dense(1, activation='sigmoid')
])

# Kompilace modelu
model.compile(
    optimizer='adam',                    # Optimalizační algoritmus
    loss='binary_crossentropy',          # Loss funkce pro binární klasifikaci
    metrics=['accuracy']                 # Metrika pro sledování
)

# Vybudování modelu s konkrétním tvarem vstupu
model.build(input_shape=(None, MAX_LENGTH))

# Zobrazení architektury
print("📐 Architektura modelu:")
model.summary()

## 6. Trénování modelu

Nyní budeme model trénovat. Použijeme:
- **EarlyStopping** — zastaví trénování, když se model přestane zlepšovat
- **Validation data** — pro sledování, jak model generalizuje

In [ ]:
# Nastavení trénování
EPOCHS = 5              # Počet průchodů celým datasetem
BATCH_SIZE = 256        # Počet vzorků v jedné dávce

# Ukládání modelu - nejprve lokální, potom Google Drive jako fallback
model_dir = 'models'
print("📁 Výchozí adresář: lokální disk (models/)")

if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        model_dir = '/content/drive/My Drive/xdeep-sentiment/models'
        print("✅ Google Drive úspěšně připojen - bude se používat jako fallback")
    except Exception as e:
        print(f"⚠️ Nepodařilo se připojit Google Drive: {e}")
        print("📁 Budu používat lokální adresář: models/")
else:
    print("💻 Neběžím v Colab")
    
import os
os.makedirs(model_dir, exist_ok=True)
print(f"📁 Modely se budou ukládat do: {model_dir}")

# Early stopping - zastaví trénování pokud se val_loss nezlepší 2 epochy za sebou
early_stop = EarlyStopping(
    monitor='val_loss',      # Sledujeme validation loss
    patience=2,              # Počet epoch bez zlepšení před zastavením
    restore_best_weights=True # Vrátí váhy z nejlepší epochy
)

# ModelCheckpoint - automaticky ukládá model po každé epoše
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    f'{model_dir}/sentiment_lstm_model.keras',
    monitor='val_loss',
    save_best_only=True,     # Uloží pouze nejlepší model
    verbose=1
)

print(f"📊 Nastavení trénování:")
print(f"   Počet epoch: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Train vzorků: {len(X_train):,}")
print(f"   Kroků na epochu: {len(X_train) // BATCH_SIZE:,}")


In [ ]:
# Trénování modelu

print("🚀 Začínám trénování...")
print("=" * 50)

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, checkpoint],  # Přidán checkpoint!
    verbose=1
)

print("=" * 50)
print("✅ Trénování dokončeno!")

## 7. Vyhodnocení a vizualizace

Podíváme se, jak model trénoval a jak si vede na testovacích datech.

In [ ]:
# Vizualizace průběhu trénování
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Graf Loss (ztráta)
axes[0].plot(history.history['loss'], label='Train Loss', marker='o')
axes[0].plot(history.history['val_loss'], label='Validation Loss', marker='o')
axes[0].set_title('Průběh Loss během trénování')
axes[0].set_xlabel('Epocha')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Graf Accuracy (přesnost)
axes[1].plot(history.history['accuracy'], label='Train Accuracy', marker='o')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
axes[1].set_title('Průběh Accuracy během trénování')
axes[1].set_xlabel('Epocha')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Výpis nejlepších hodnot
best_epoch = np.argmin(history.history['val_loss']) + 1
print(f"\n📊 Nejlepší epocha: {best_epoch}")
print(f"   Val Loss: {history.history['val_loss'][best_epoch-1]:.4f}")
print(f"   Val Accuracy: {history.history['val_accuracy'][best_epoch-1]:.4f}")

In [ ]:
# Vyhodnocení modelu na testovacích datech
print("🧪 Vyhodnocuji model na testovacích datech...")
print("=" * 50)

# Predikce na testovacích datech
y_pred_prob = model.predict(X_test, verbose=0)  # Pravděpodobnosti (0-1)

# Diagnostika - podíváme se na rozložení predikcí
print(f"\n🔍 Diagnostika predikcí:")
print(f"   Min pravděpodobnost: {y_pred_prob.min():.6f}")
print(f"   Max pravděpodobnost: {y_pred_prob.max():.6f}")
print(f"   Průměr: {y_pred_prob.mean():.6f}")
print(f"   Prvních 10 predikcí: {y_pred_prob[:10].flatten()}")

y_pred = (y_pred_prob > 0.5).astype(int).flatten()  # Převod na 0/1

print(f"\n   Počet predikovaných 0 (negativní): {(y_pred == 0).sum():,}")
print(f"   Počet predikovaných 1 (pozitivní): {(y_pred == 1).sum():,}")
print(f"   Skutečných 0: {(y_test == 0).sum():,}")
print(f"   Skutečných 1: {(y_test == 1).sum():,}")

# Výpočet metrik
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\n📊 Metriky na testovacích datech:")
print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   Precision: {precision:.4f}")
print(f"   Recall:    {recall:.4f}")
print(f"   F1-Score:  {f1:.4f}")

print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Negativní', 'Pozitivní']))

In [ ]:
# Pokus o načtení uloženého modelu (pokud existuje)
import os

# Použijeme stejnou cestu jako při trénování
if IN_COLAB:
    model_path = '/content/drive/My Drive/xdeep-sentiment/models/sentiment_lstm_model.keras'
else:
    model_path = 'models/sentiment_lstm_model.keras'

if os.path.exists(model_path):
    print(f"✅ Nalezen uložený model: {model_path}")
    model = tf.keras.models.load_model(model_path)
    print("✅ Model úspěšně načten!")

    model.summary()    
    print("⚠️ Spusť nejprve trénování (buňka 28).")

else:    print(f"❌ Model nebyl nalezen: {model_path}")


In [ ]:
# Vizualizace Confusion Matrix (matice záměn)
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(cm, cmap='Blues')

# Popisky
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Negativní', 'Pozitivní'])
ax.set_yticklabels(['Negativní', 'Pozitivní'])
ax.set_xlabel('Predikce')
ax.set_ylabel('Skutečnost')
ax.set_title('Confusion Matrix')

# Přidání hodnot do buněk
for i in range(2):
    for j in range(2):
        text = ax.text(j, i, f'{cm[i, j]:,}', ha='center', va='center', 
                       color='white' if cm[i, j] > cm.max()/2 else 'black', fontsize=14)

plt.colorbar(im)
plt.tight_layout()
plt.show()

# Interpretace
print(f"\n📊 Interpretace Confusion Matrix:")
print(f"   ✅ True Negatives (správně negativní):  {cm[0,0]:,}")
print(f"   ❌ False Positives (falešně pozitivní): {cm[0,1]:,}")
print(f"   ❌ False Negatives (falešně negativní): {cm[1,0]:,}")
print(f"   ✅ True Positives (správně pozitivní):  {cm[1,1]:,}")

## 8. Inference pro nové texty

Vytvoříme funkci, která umožní predikovat sentiment pro libovolný nový text.

In [ ]:
def predict_sentiment(text, model=model, tokenizer=tokenizer, max_length=MAX_LENGTH):
    """
    Predikuje sentiment pro daný text.
    
    Args:
        text: Text k analýze (string)
        model: Natrénovaný model
        tokenizer: Tokenizer použitý při trénování
        max_length: Maximální délka sekvence
        
    Returns:
        dict s predikcí, pravděpodobností a interpretací
    """
    # Vyčištění textu (stejná funkce jako při trénování)
    cleaned = clean_text(text)
    
    # Tokenizace
    sequence = tokenizer.texts_to_sequences([cleaned])
    
    # Padding - MUSÍ být stejný jako při trénování (padding='pre')
    padded = pad_sequences(sequence, maxlen=max_length, padding='pre', truncating='post')
    
    # Predikce
    probability = model.predict(padded, verbose=0)[0][0]
    sentiment = 'Pozitivní 😊' if probability > 0.5 else 'Negativní 😠'
    confidence = probability if probability > 0.5 else 1 - probability
    
    return {
        'text': text,
        'cleaned': cleaned,
        'sentiment': sentiment,
        'probability': probability,
        'confidence': confidence * 100
    }

print("✅ Funkce predict_sentiment() je připravena!")

In [ ]:
# Testování na vlastních textech
test_texts = [
    "I absolutely love this product! Best purchase ever!",
    "This is terrible. Waste of money. Never buying again.",
    "The weather is nice today.",
    "I'm so excited about the weekend trip!",
    "My flight was cancelled and I'm stuck at the airport.",
    "Just had an amazing dinner with friends!",
    "This movie was boring and predictable.",
    "This was the best night of my life. But on the other hand, it was the worst night of my life."
]

print("🔮 Testování modelu na nových textech:")
print("=" * 60)

for text in test_texts:
    result = predict_sentiment(text)
    print(f"\n📝 \"{text[:50]}{'...' if len(text) > 50 else ''}\"")
    print(f"   → {result['sentiment']} (jistota: {result['confidence']:.1f}%)")

## 9. Uložení modelu

Uložíme natrénovaný model pro pozdější použití.

In [ ]:
# Uložení modelu a tokenizeru (na Google Drive pro perzistenci)
import pickle
import os

# Použijeme stejnou cestu jako při trénování
if IN_COLAB:
    save_dir = '/content/drive/My Drive/xdeep-sentiment/models'
else:
    save_dir = 'models'
os.makedirs(save_dir, exist_ok=True)

# Uložení Keras modelu (nativní formát)
model_path = os.path.join(save_dir, 'sentiment_lstm_model.keras')
model.save(model_path)
print(f"✅ Model uložen: {model_path}")

# Uložení tokenizeru (potřebujeme ho pro inference)
tokenizer_path = os.path.join(save_dir, 'tokenizer.pkl')
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)
print(f"✅ Tokenizer uložen: {tokenizer_path}")

# Uložení konfigurace
config = {
    'VOCAB_SIZE': VOCAB_SIZE,
    'MAX_LENGTH': MAX_LENGTH,
    'EMBEDDING_DIM': EMBEDDING_DIM
}
config_path = os.path.join(save_dir, 'config.pkl')
with open(config_path, 'wb') as f:
    pickle.dump(config, f)

print(f"✅ Konfigurace uložena: {config_path}")
print("💡 Tyto soubory přežijí restart Colab runtime!")

print(f"\n📁 Všechny soubory jsou na Google Drive: {save_dir}/")


## Shrnutí

Úspěšně jsme vytvořili model pro analýzu sentimentu! 

**Co jsme se naučili:**
- Načítání a předzpracování textových dat
- Tokenizace a padding sekvencí
- Vytvoření a trénování LSTM modelu
- Vyhodnocení pomocí metrik a confusion matrix
- Inference pro nové texty

**Možná vylepšení:**
- Použít `padding='pre'` místo `'post'` (lepší pro LSTM)
- Přidat `Bidirectional(LSTM(...))` pro obousměrné zpracování
- Experimentovat s jinými architekturami (GRU, Transformer)
- Fine-tuning na specifické doméně